### Optimizing Temporary Housing Deployment to Reduce U.S. Homelessness 
Nauman Sohani, Yannan Tuo, Charlie Nitschelm

In [ ]:
# Load packages
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [1]:
# Import datasets
# from <data_source> import *
%whos

Interactive namespace is empty.


In [ ]:
# Set up Gurobi environment
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()

# Initialize the model
m = gp.Model(env=env)

In [ ]:
# High level variables + assumptions

In [ ]:
# Decision variables

In [ ]:
# Constraints

In [ ]:
# Objective function

In [ ]:
# Update and write the model
m.update() # Update model parameters
m.write("uplift.lp") # Write model to file

# Solve
m.optimize()

# Check status != 3 (3 is infeasible)
# print(m.status)


#### Analysis of optimization results
beep boop